## LDA Topic Modeling and Mapping Topics

The goal is to complete topic modeling on my small sample of abstracts and explore how I can work towards mapping topics across the world

In [24]:
import pandas as pd

from nltk.corpus import brown
from nltk.corpus import stopwords

import numpy as np
import pandas as pd

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import LdaModel
from gensim.corpora import Dictionary

from gensim.models.ldamulticore import LdaMulticore
from tqdm import trange, notebook

import pyLDAvis
import pyLDAvis.gensim

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS

from pprint import pprint
from collections import Counter, defaultdict

nlp = spacy.load('en_core_web_sm')
sw = stopwords.words("english")

In [25]:
##Loading data
data=pd.read_csv("WoS_Data.csv")
data

,Article Title,Source Title,Author Keywords,Abstract,Addresses,"Times Cited, All Databases",First_A
0,Land enrolled in the Conservation Reserve Prog...,GLOBAL ECOLOGY AND CONSERVATION,Conservation Reserve Program; Discrete choice ...,All animals must select sites to rest and may ...,"[Tanner, Evan P.; Tanner, Ashley M.] Texas A&M...",0,USA
1,Fish Biodiversity Conservation and Restoration...,WATER,conservation; ecological integrity; fish biodi...,Although many significant policies and legisla...,"[Li, Jianhua] Tongji Univ, Key Lab Yangtze Wat...",2,Peoples R China
2,Operationalizing ecological connectivity in sp...,METHODS IN ECOLOGY AND EVOLUTION,decision support software; demographic; ecolog...,"Globally, protected areas are being establishe...","[Daigle, Remi M.; Metaxas, Anna; Balbar, Ariea...",25,Canada
3,Spatial conservation planning with ecological ...,BIOLOGICAL CONSERVATION,Conservation feedbacks; Dynamic spatial priori...,Most spatial conservation prioritisations bein...,"[Larrosa, Cecilia; Carrasco, Luis R.] Natl Uni...",2,Singapore
4,A landscape index of ecological integrity to i...,LANDSCAPE ECOLOGY,Landscape pattern; Landscape metrics; Ecologic...,Conservation planning is increasingly using co...,"[McGarigal, Kevin; Compton, Bradley W.; Plunke...",16,USA
...,...,...,...,...,...,...,...
995,Disturbance Ecology in the Anthropocene,FRONTIERS IN ECOLOGY AND EVOLUTION,anthropogenic change; disturbance ecology; dis...,With the accumulating evidence of changing dis...,"[Newman, Erica A.] Univ Arizona, Dept Ecol & E...",34,USA
996,Integrating socio-ecological dynamics into lan...,LAND USE POLICY,Tree farms; Socio-Ecological; Dyna-CLUE; InVES...,Chile is one of the first documented nations t...,"[Manuschevich, Daniela] Univ Acad Humanismo Cr...",8,Chile
997,Persistence through tough times: fixed and shi...,BIODIVERSITY AND CONSERVATION,Endangered species; Biodiversity conservation;...,It may be possible to avert threatened species...,"[Reside, April E.; Fisher, Diana O.] Univ Quee...",23,Australia
998,Can local ecological knowledge provide meaning...,OCEAN & COASTAL MANAGEMENT,Hainan; Indo-Pacific humpback dolphin; Questio...,Identifying and evaluating potentially suitabl...,"[Lin, Mingli; Xing, Luru; Fang, Liang; Li, Son...",11,Peoples R China


In [32]:
# # Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
     return [[word for word in simple_preprocess(str(doc)) if word not in sw] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations


def display_topics(model, features, no_top_words=5):
    for topic, word_vector in enumerate(model.components_):
        total = word_vector.sum()
        largest = word_vector.argsort()[::-1] # invert sort order
        print("\nTopic %02d" % topic)
        for i in range(0, no_top_words):
            print(" %s (%2.2f)" % (features[largest[i]],
                                   word_vector[largest[i]]*100.0/total))



1000

In [31]:
abstract=data['Abstract']

word_dat=[] ##creating a list to store string of data

for item in abstract:  #Make everything in the column  a string of text
    string=str(item)
    word_dat.append(string)
    
len(word_dat)   
word_dat[1]

'Although many significant policies and legislation have been put forth by the Chinese government aiming toward ecological civilization for the aquatic environment, in practice, the situation is still undesirable. A pioneering multi-year study has since been conducted on the East Tiaoxi River (a major tributary of the renowned Yangtze River), focusing on fish distribution, influencing factors and habitat requirements, and river health as well. This revealed certain key species, hotspot protection areas, negative impacts of local anthropogenic activities, and inappropriate perceptions and practices of conservation and restoration. We found that not all fish species were equally conserved and that the supposedly ecological engineering measures were initiated without regard for ecological integrity. Our exemplary study appreciates scientific basis and truly ecological notion, and urgently advocates comprehensive and continuous basin-scale scientific investigation of fish biodiversity, and

In [ ]:
data_words = list(sent_to_words(word_dat))

len(data_words)

# Topic Modeling

In [33]:
## Working with bigrams and trigrams

bigram2 = gensim.models.Phrases(data_words, min_count=5,threshold=100)
trigram = gensim.models.Phrases(bigram2[data_words],threshold=100)

bigram_mod= gensim.models.phrases.Phraser(bigram2)

trigram_mod=gensim.models.phrases.Phraser(trigram)

 #trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['all', 'animals', 'must', 'select', 'sites', 'to', 'rest', 'and', 'may', 'spend', 'large', 'portion', 'of', 'their', 'lives', 'doing_so', 'despite', 'the', 'importance', 'of', 'this', 'period', 'in', 'their', 'daily', 'activity', 'budget', 'we', 'lack', 'information', 'about', 'rest', 'roost', 'ecology', 'for', 'most', 'animals', 'including', 'the', 'imperiled', 'lesser_prairie_chicken', 'tympanuchus', 'pallidicinctus', 'hereafter', 'lepc', 'therefore', 'we', 'sought', 'to', 'identify', 'how', 'landcover', 'anthropogenic', 'features', 'and', 'human', 'policy', 'presence', 'of', 'the', 'conservation', 'reserve', 'program', 'crp', 'influenced', 'roost', 'site_selection', 'and', 'movement', 'patterns', 'of', 'the', 'lepc', 'from', 'march', 'to', 'may', 'we', 'captured', 'and', 'fitted', 'gps', 'transmitters', 'to', 'lepcs', 'males', 'females', 'within', 'beaver', 'county', 'oklahoma', 'and', 'recorded', 'two', 'nocturnal', 'locations', 'per', 'hour', 'period', 'annually', 'we', 'used', '

In [34]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)


#data_words_bigrams[0]

In [35]:
# # Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


print(data_lemmatized[0])

['animal', 'must', 'select', 'site', 'rest', 'may', 'spend', 'large', 'portion', 'life', 'importance', 'period', 'daily', 'activity', 'budget', 'lack', 'information', 'rest', 'roost', 'ecology', 'animal', 'include', 'imperil', 'chicken', 'pallidicinctus', 'hereafter', 'lepc', 'therefore', 'seek', 'identify', 'landcover', 'feature', 'human', 'policy', 'presence', 'conservation', 'reserve', 'program', 'roost', 'movement', 'pattern', 'lepc', 'may', 'capture', 'fit', 'transmitter', 'male', 'female', 'record', 'nocturnal', 'location', 'hour', 'period', 'annually', 'use', 'discrete', 'choice', 'model', 'generalized_linear', 'mixed', 'effect', 'model', 'determine', 'cover', 'patch', 'feature', 'influence', 'roost', 'movement', 'roosting', 'site', 'respectively', 'find', 'roost', 'site', 'close', 'crp', 'cropland', 'would', 'expect', 'random', 'conversely', 'roost', 'site', 'locate', 'farther', 'away', 'shortgrass', 'prairie', 'road', 'transmission', 'line', 'expect', 'probability', 'lepc', 'r

In [36]:
dict_gensim_abs = Dictionary(data_lemmatized)

In [37]:
#words must appear in at least 5 documents and no more than 80%
dict_gensim_abs.filter_extremes(no_below=5, no_above=0.7)

In [38]:
#Calculates bag of words matrix
bow_gensim_abs = [dict_gensim_abs.doc2bow(w) \
                   for w in data_lemmatized]

In [39]:
#TF-IDF transformation
from gensim.models import TfidfModel
tfidf_gensim_abs = TfidfModel(bow_gensim_abs)
vectors_gensim_abs = tfidf_gensim_abs[bow_gensim_abs]

## Identifying Optimal Number of Topics

In [ ]:
##Coherence for topics 3 through 14

lda_para_model_n = []
for n in trange(3, 14):
    lda_model = LdaMulticore(corpus=bow_gensim_abs, id2word=dict_gensim_abs,
                             chunksize=2000, eta='auto', iterations=400,
                             num_topics=n, passes=100, eval_every=None,
                             random_state=42)
    lda_coherence = CoherenceModel(model=lda_model, texts=data_lemmatized,
                                   dictionary=dict_gensim_abs, coherence='c_v')
    lda_para_model_n.append((n, lda_model, lda_coherence.get_coherence()))

In [ ]:
pd.DataFrame(lda_para_model_n, columns=["n", "model", \
"coherence"]).set_index("n")[["coherence"]].plot(figsize=(16,9))

Since there are peaks at 7 topics and 12 topics, I will be visualizing both of these to see more about the differences and similarities between them

## Visualizing at 7 and 12 topics

In [40]:
lda_7_topics= LdaModel(corpus=bow_gensim_abs, id2word=dict_gensim_abs,
                       chunksize=2000, alpha='auto', eta='auto', iterations=400, num_topics=7,
                       passes=100, eval_every=None, random_state=42)
lda_12_topics= LdaModel(corpus=bow_gensim_abs, id2word=dict_gensim_abs,
                       chunksize=2000, alpha='auto', eta='auto', iterations=400, num_topics=12,
                       passes=100, eval_every=None, random_state=42)

In [ ]:
lda_7_topics.show_topics()

In [ ]:
lda_12_topics.show_topics()

In [ ]:
## Topic Wordclouds

import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
count_para_vectorizer = CountVectorizer(stop_words=stopwords, min_df=5,
max_df=0.7)
def wordcloud_topics(model, features, no_top_words=40):
    for topic, words in enumerate(model.components_):
        size = {}
        largest = words.argsort()[::-1] # invert sort order
        for i in range(0, no_top_words):
            size[features[largest[i]]] = abs(words[largest[i]])
            wc = WordCloud(background_color="white", max_words=100,
                           width=960, height=540)
            wc.generate_from_frequencies(size)
            plt.figure(figsize=(12,12))
            plt.imshow(wc, interpolation='bilinear')
            plt.axis("off")
            # if you don't want to save the topic model, comment the next line
            plt.savefig(f'topic{topic}.png')

In [ ]:
#Wordclouds for 7 topics
for t in range(lda_7_topics.num_topics):
    plt.figure()
    plt.imshow(WordCloud().fit_words(dict(lda_7_topics.show_topic(t, 20))))
    plt.axis("off")
    plt.title("Topic #" + str(t))
    plt.show()

In [ ]:
#Wordclouds for 12 topics
for t in range(lda_12_topics.num_topics):
    plt.figure()
    plt.imshow(WordCloud().fit_words(dict(lda_12_topics.show_topic(t, 20))))
    plt.axis("off")
    plt.title("Topic #" + str(t))
    plt.show()

# Investigating Topic Distribution

In [41]:
def format_topics_sentences(ldamodel=lda_7_topics, corpus=bow_gensim_abs, texts=word_dat):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_7_topics, corpus=bow_gensim_abs, texts=word_dat)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)
dominant_topic_sample=df_dominant_topic.head(10)

In [30]:
#Making sure this exports correctly

dominant_topic_sample.to_csv(r'C:\Users\maddo\Desktop\Dominant_Topic_Sample.csv', index=False)

In [34]:
df_dominant_topic.to_csv(r'C:\Users\maddo\Desktop\Dominant_Topic_Dataframe.csv', index=False)

In [42]:
#making dominant topics a list and placing in WoS data

dominant_topics=df_dominant_topic['Dominant_Topic']
data['Dominant_Topic']=dominant_topics
data

,Article Title,Source Title,Author Keywords,Abstract,Addresses,"Times Cited, All Databases",First_A,Dominant_Topic
0,Land enrolled in the Conservation Reserve Prog...,GLOBAL ECOLOGY AND CONSERVATION,Conservation Reserve Program; Discrete choice ...,All animals must select sites to rest and may ...,"[Tanner, Evan P.; Tanner, Ashley M.] Texas A&M...",0,USA,2.0
1,Fish Biodiversity Conservation and Restoration...,WATER,conservation; ecological integrity; fish biodi...,Although many significant policies and legisla...,"[Li, Jianhua] Tongji Univ, Key Lab Yangtze Wat...",2,Peoples R China,3.0
2,Operationalizing ecological connectivity in sp...,METHODS IN ECOLOGY AND EVOLUTION,decision support software; demographic; ecolog...,"Globally, protected areas are being establishe...","[Daigle, Remi M.; Metaxas, Anna; Balbar, Ariea...",25,Canada,1.0
3,Spatial conservation planning with ecological ...,BIOLOGICAL CONSERVATION,Conservation feedbacks; Dynamic spatial priori...,Most spatial conservation prioritisations bein...,"[Larrosa, Cecilia; Carrasco, Luis R.] Natl Uni...",2,Singapore,1.0
4,A landscape index of ecological integrity to i...,LANDSCAPE ECOLOGY,Landscape pattern; Landscape metrics; Ecologic...,Conservation planning is increasingly using co...,"[McGarigal, Kevin; Compton, Bradley W.; Plunke...",16,USA,1.0
...,...,...,...,...,...,...,...,...
995,Disturbance Ecology in the Anthropocene,FRONTIERS IN ECOLOGY AND EVOLUTION,anthropogenic change; disturbance ecology; dis...,With the accumulating evidence of changing dis...,"[Newman, Erica A.] Univ Arizona, Dept Ecol & E...",34,USA,0.0
996,Integrating socio-ecological dynamics into lan...,LAND USE POLICY,Tree farms; Socio-Ecological; Dyna-CLUE; InVES...,Chile is one of the first documented nations t...,"[Manuschevich, Daniela] Univ Acad Humanismo Cr...",8,Chile,1.0
997,Persistence through tough times: fixed and shi...,BIODIVERSITY AND CONSERVATION,Endangered species; Biodiversity conservation;...,It may be possible to avert threatened species...,"[Reside, April E.; Fisher, Diana O.] Univ Quee...",23,Australia,2.0
998,Can local ecological knowledge provide meaning...,OCEAN & COASTAL MANAGEMENT,Hainan; Indo-Pacific humpback dolphin; Questio...,Identifying and evaluating potentially suitabl...,"[Lin, Mingli; Xing, Luru; Fang, Liang; Li, Son...",11,Peoples R China,2.0


In [43]:
##Everything looks good
##Now lets update my WoS data

data.to_csv(r'C:\Users\maddo\Desktop\WoS_Data.csv', index=False)

# Geographically Mapping Topics

https://towardsdatascience.com/when-topic-modeling-is-part-of-the-text-pre-processing-294b58d35514#:~:text=Create%20document-topic%20matrix.%20Create%20a%20data%20frame%20where,of%20each%20topic%20is%20assigned%20to%20each%20document.

https://www.machinelearningplus.com/nlp/topic-modeling-python-sklearn-examples/#14howtoseethedominanttopicineachdocument

https://plotly.com/python/bubble-maps/ (This contains what I ended up using)

In [ ]:
import geopandas as gpd
import geoplot as gplt
import pygal
import pygal_maps_world
import numpy as np
import plotly
import plotly.express as px

In [44]:
topics_dataframe=pd.DataFrame()

topics_dataframe['Country']=data['First_A']
topics_dataframe['Dominant_Topic']=data['Dominant_Topic']

In [98]:
topics_dataframe

,Country,Dominant_Topic
0,USA,2.0
1,Peoples R China,3.0
2,Canada,1.0
3,Singapore,1.0
4,USA,1.0
...,...,...
995,USA,0.0
996,Chile,1.0
997,Australia,2.0
998,Peoples R China,2.0


In [96]:
topics_dataframe['Dominant_Topic'] = pd.Categorical(topics_dataframe.Dominant_Topic)
topics_dataframe['Country'] = pd.Categorical(topics_dataframe.Country)

In [97]:
#topics_dataframe.dtypes

Country           category
Dominant_Topic    category
dtype: object

In [104]:


topic_count_by_country=topics_dataframe.groupby( [ "Country", "Dominant_Topic"] ).size().to_frame(name = 'count').reset_index()

In [106]:
#inspect a csv

topic_count_by_country.to_csv(r'C:\Users\maddo\Desktop\topic_count_by_country.csv', index=False)

In [119]:
topic_count_by_country

,Country,Dominant_Topic,count
0,Algeria,2.0,1
1,Argentina,0.0,1
2,Argentina,2.0,10
3,Argentina,3.0,1
4,Argentina,6.0,1
...,...,...,...
220,USA,5.0,21
221,USA,6.0,56
222,Uruguay,3.0,1
223,Zimbabwe,0.0,2


In [122]:
topic_count_by_country.dtypes

Country           category
Dominant_Topic    category
count                int64
dtype: object

In [123]:
##Renaming to approximate topic labels (subjectively assigned based on keywords, texts assigned a topic, and knowledge of field)

topic_count_by_country['Dominant_Topic'] = topic_count_by_country['Dominant_Topic'].replace(
    to_replace=[0.0, 1.0,2.0,3.0,4.0,5.0,6.0], 
    value=['Land Use, Forests', 'Land Use, Urban','Habitat Use, Distribution',
           'Marine Area Protection, Management', 'Human-Wildlife Interface','Local Knowledge, Communities',
           'Social Research'])
print(topic_count_by_country)

       Country                      Dominant_Topic  count
0      Algeria           Habitat Use, Distribution      1
1    Argentina                   Land Use, Forests      1
2    Argentina           Habitat Use, Distribution     10
3    Argentina  Marine Area Protection, Management      1
4    Argentina                     Social Research      1
..         ...                                 ...    ...
220        USA        Local Knowledge, Communities     21
221        USA                     Social Research     56
222    Uruguay  Marine Area Protection, Management      1
223   Zimbabwe                   Land Use, Forests      2
224   Zimbabwe        Local Knowledge, Communities      1

[225 rows x 3 columns]


# Bubble Maps
https://plotly.com/python/bubble-maps/

In [126]:
#The goal of this map is to toggle between model topics and see where articles assigned to the topic are published

import plotly.express as px
#df = px.dataframe.gapminder()
fig = px.scatter_geo(topic_count_by_country, locations='Country',locationmode='country names', #color='Country',
                     hover_name='Country', size='count', animation_group='Dominant_Topic',animation_frame='Dominant_Topic',
                    title='Dominant Topics by Country')
                   
fig.show()

In [127]:
#Saving
fig.write_html("Draft_Topics_By_Country_2.html")